In [1]:
import pandas as pd
import numpy as np
import seaborn as sb
import matplotlib.pyplot as plt
%matplotlib inline
import re

# Analysis
Para mais detalhes dos experimentos consulte os relatório técnico.

In [7]:
file = "./output_clustering/OCoClusResult_Rs2020.txt"
df_fimi = pd.read_csv(file, header=None, names=["transation"])
columnClusters,rowClusters = processDocument(df_fimi)
del df_fimi
check_result(columnClusters,rowClusters,rs)

IndexError: list index out of range

# Resultado filtrado ococlus

In [ ]:
file = "C:/Users/yurin/Documents/my_github/doutorado/pesquisa/datasets/covid19/OCoClusResult_CV19_clean.txt"
df_fimi = pd.read_csv(file, header=None, names=["transation"])
columnClusters,rowClusters = processDocument(df_fimi)
del df_fimi
check_result(columnClusters,rowClusters,municipio_regiao_covid)

In [ ]:
file = "C:/Users/yurin/Documents/my_github/doutorado/pesquisa/datasets/covid19/OCoClusResult_CV19_c2.txt"
df_fimi = pd.read_csv(file, header=None, names=["transation"])
# columnClusters,rowClusters = processDocument(df_fimi)
# del df_fimi
# check_result(columnClusters,rowClusters,municipio_regiao_covid)

# Support methods

In [2]:
def check_result(columnClusters,rowClusters,map_att_dict):
    nc = len(columnClusters)
    for i in range(nc):
        num_objects = len(set(rowClusters[i]))
        print("Cluster-"+str(i+1)+" ("+str(num_objects)+")")
        print("Atributos: ",end="")
        for ele in columnClusters[i]:
            print(map_att_dict[int(ele)],end=", ")

        print("\nCompartilhamento de OBJETOS em outros clusters.")
        for j in range(nc):
            if i != j:
                print("    Cluster "+str(j+1)+": ",end="")
#                 clusB = len(set(rowClusters[i]))
                qtd_intersect = len(set(rowClusters[j]).intersection(set(rowClusters[i])))
#                 overlap = ((num_objects-(num_objects-qtd_intersect))/len(set(rowClusters[i]).union(set(rowClusters[j]))))*100
                overlap = ((num_objects-(num_objects-qtd_intersect))/num_objects)*100
#                 overlap = ((cluSize-(cluSize-qtd_intersect)))*100
                print("{0:.2f}% ({1:d})".format(overlap,qtd_intersect),end="")
                print("| [",end="")
                for attribute_idx in columnClusters[j]:
                    print(map_att_dict[int(attribute_idx)],end=", ")
                print("]", end=" ")
                print("QTD pacientes: "+str(len(rowClusters[j])))
        print(" ")

In [3]:
def processDocument(file):
    #print("Executing processDocument() method.")
    size_df = file.shape
    uniqueP = []
    uniqueT = []
    pois_per_cluster = []
    trajs_per_cluster = []
    change = 0
    
    for i in range(size_df[0]):#number of transations
        transationString = file.transation[i].split(' ')
#         print(transationString)
        pois = []
        trajs = []
        for j in range(len(transationString)):
            try:
                tmp = int(transationString[j])
            except:
                tmp = ''
            
            if change == 0:
                if type(tmp) is int:
                    if transationString[j] not in uniqueP:
                        uniqueP.append(transationString[j])
#                         if not pois: # empty
                    pois.append(transationString[j])
#                         print(transationString[j],end="|")
                else:
    #                 print("NO",end=" | ")
                    change = 1
#                     print("")
            else:
                if type(tmp) is int:
                    if transationString[j] not in uniqueT:
                        uniqueT.append(transationString[j])
                    trajs.append(transationString[j])
                else:
    #                 print("NO",end=" | ")
                    if change == 1:
                        change = 2
                        tmp = re.split('\[| |\]',transationString[j])
                        #print("1-tmp:"+str(tmp))
                        if tmp[1] not in uniqueT:
                            uniqueT.append(tmp[1])
                        trajs.append(tmp[1])
                    else:
                        tmp = re.split('\[| |\]',transationString[j])
                        #print("2-tmp:"+str(tmp))
                        if tmp[0] not in uniqueT:
                            uniqueT.append(tmp[0])
                        trajs.append(tmp[0])
                        change = 0
        pois_per_cluster.append(pois)
        trajs_per_cluster.append(trajs)

#     print('Number of unique elements: ',len(uniqueElements))
#     print("P per Clusters: "+str(pois_per_cluster))
#     print("T per Clusters: "+str(trajs_per_cluster))
    return pois_per_cluster,trajs_per_cluster

In [ ]:
def chord_vis(colClus,rowClus):
# Só da para ver até 20 cores    
    num_of_cluster = len(colClus)
    nodes = []
    for i in range(num_of_cluster):
        nodes.append('G'+str(i+1))
    print("New nodes: ",nodes)
    
    links = np.zeros((num_of_cluster,num_of_cluster),dtype=int)
    print("Size links:",links.shape)
    
    for i in range(num_of_cluster):
        tmp = set(rowClus[i])
        for j in range(num_of_cluster):
            if i != j:
                obj_intenserction_qtd = len(set(rowClus[i]).intersection(set(rowClus[j])))
                if obj_intenserction_qtd != 0:
                    links[i][j] = obj_intenserction_qtd
            else:
                for z in range(num_of_cluster):
                    if i != z:
                        tmp.difference_update(rowClus[z])
                if len(tmp) > 0:
                    links[i][j] = len(tmp)
    
    fig = plt.figure(figsize=(6,6))
    ax = plt.axes([0,0,1,1])
    nodePos = chordDiagram(links, ax)
    ax.axis('off')
#     prop = dict(fontsize=16*0.8, ha='center', va='center')
    prop = dict(fontsize=16*0.8, ha='center', va='center')
    for i in range(num_of_cluster):
        ax.text(nodePos[i][0], nodePos[i][1], nodes[i], rotation=nodePos[i][2], **prop)

In [ ]:
heatmap_vis(columnClusters,rowClusters)

In [ ]:
heatmap_accumulative_vis(columnClusters,rowClusters)

In [ ]:
def heatmap_vis(colClus,rowClus):
    f, axes = plt.subplots(figsize=(4,5.5))
    num_of_cluster = len(colClus)
    nodes = []
    for i in range(num_of_cluster):
        nodes.append('G'+str(i+1))
#     print("New nodes: ",nodes)
    
    links = np.zeros((num_of_cluster,num_of_cluster),dtype=int)
#     print("Size links:",links.shape)
    
    for i in range(num_of_cluster):
        tmp = set(rowClus[i])
        for j in range(num_of_cluster):
            if i != j:
                obj_intenserction_qtd = len(set(rowClus[i]).intersection(set(rowClus[j])))
                if obj_intenserction_qtd != 0:
                    links[i][j] = obj_intenserction_qtd
            else:
                for z in range(num_of_cluster):
                    if i != z:
                        tmp.difference_update(rowClus[z])
                if len(tmp) > 0:
                    links[i][j] = len(tmp)

    # Generate a mask for the upper triangle
    mask = np.triu(np.ones_like(links, dtype=bool))
    
    # Generate a custom diverging colormap
#     cmap = sb.diverging_palette(220, 20, n=10, as_cmap=True)
    cmap = sb.diverging_palette(240, 10, n=11, as_cmap=True)
    
#     ax = sb.heatmap(m1, xticklabels=2, yticklabels=False)
    #colors: [coolwarm,Reds]
    ax = sb.heatmap(links,xticklabels=nodes,yticklabels=nodes,cmap="Reds",
#     ax = sb.heatmap(links,mask=mask,xticklabels=nodes,yticklabels=nodes,cmap="Reds",
#     ax = sb.heatmap(links,mask=mask,xticklabels=nodes,yticklabels=nodes,cmap=cmap,
                    cbar_kws={"orientation": "horizontal",'label': 'Number of sharable objects'})
#     ax.set_yticklabels(ax.get_yticklabels(), rotation=30)

In [ ]:
def heatmap_accumulative_vis(colClus,rowClus):
    '''
    Essa visualização considera o número de elementos compartilhados do cluster referência i no acumulativo sequencial
    com o cluster j.
    Exemplo:
    grupos-> g1,g2 e g3
    g1 = g1 - g1.intersection.g2_g3
    g2 = g2.intersection.g1
    g3 = g3.intersection.g1_g2
    '''
    f, axes = plt.subplots(figsize=(4,5.5))
    num_of_cluster = len(colClus)
    nodes = []
    for i in range(num_of_cluster):
        nodes.append('G'+str(i+1))
    
    links = np.zeros((num_of_cluster,num_of_cluster),dtype=int)
    
    for i in range(num_of_cluster):
        tmp = set(rowClus[i])
        obj_intenserction_qtd = set(rowClus[i])
        tmp_acc = obj_intenserction_qtd.copy()
        for j in range(num_of_cluster):
            if i != j:
#                 tmp_acc = obj_intenserction_qtd.copy()
                obj_intenserction_qtd = obj_intenserction_qtd.intersection(set(rowClus[j]))
                print(i,j,end=": ")
                print(len(obj_intenserction_qtd))
                if len(obj_intenserction_qtd) != 0:
                    links[i][j] = len(obj_intenserction_qtd)
                    tmp_acc = obj_intenserction_qtd.copy()
                else:
                    obj_intenserction_qtd = tmp_acc.copy()
#             else:
#                 for z in range(num_of_cluster):
#                     if i != z:
#                         tmp.difference_update(rowClus[z])
#                 if len(tmp) > 0:
#                     links[i][j] = len(tmp)

    # Generate a mask for the upper triangle
    mask = np.triu(np.ones_like(links, dtype=bool))
    
    # Generate a custom diverging colormap
#     cmap = sb.diverging_palette(220, 20, n=10, as_cmap=True)
#     cmap = sb.diverging_palette(240, 10, n=11, as_cmap=True)
    cmap = sb.diverging_palette(240, 10, n=num_of_cluster+1, as_cmap=True)
    
#     ax = sb.heatmap(m1, xticklabels=2, yticklabels=False)
    #colors: [coolwarm,Reds]
    ax = sb.heatmap(links,xticklabels=nodes,yticklabels=nodes,cmap="Reds",
#     ax = sb.heatmap(links,mask=mask,xticklabels=nodes,yticklabels=nodes,cmap="Reds",
#     ax = sb.heatmap(links,mask=mask,xticklabels=nodes,yticklabels=nodes,cmap=cmap,
                    cbar_kws={"orientation": "horizontal",'label': 'Number of sharable objects'})
#     ax.set_yticklabels(ax.get_yticklabels(), rotation=30)

In [ ]:
###################
# chord diagram
# https://github.com/fengwangPhysics/matplotlib-chord-diagram
import matplotlib.pyplot as plt
from matplotlib.path import Path
import matplotlib.patches as patches

import numpy as np

LW = 0.3

def polar2xy(r, theta):
    return np.array([r*np.cos(theta), r*np.sin(theta)])

def hex2rgb(c):
    return tuple(int(c[i:i+2], 16)/256.0 for i in (1, 3 ,5))

def IdeogramArc(start=0, end=60, radius=1.0, width=0.2, ax=None, color=(1,0,0)):
    # start, end should be in [0, 360)
    if start > end:
        start, end = end, start
    start *= np.pi/180.
    end *= np.pi/180.
    # optimal distance to the control points
    # https://stackoverflow.com/questions/1734745/how-to-create-circle-with-b%C3%A9zier-curves
    opt = 4./3. * np.tan((end-start)/ 4.) * radius
    inner = radius*(1-width)
    verts = [
        polar2xy(radius, start),
        polar2xy(radius, start) + polar2xy(opt, start+0.5*np.pi),
        polar2xy(radius, end) + polar2xy(opt, end-0.5*np.pi),
        polar2xy(radius, end),
        polar2xy(inner, end),
        polar2xy(inner, end) + polar2xy(opt*(1-width), end-0.5*np.pi),
        polar2xy(inner, start) + polar2xy(opt*(1-width), start+0.5*np.pi),
        polar2xy(inner, start),
        polar2xy(radius, start),
        ]

    codes = [Path.MOVETO,
             Path.CURVE4,
             Path.CURVE4,
             Path.CURVE4,
             Path.LINETO,
             Path.CURVE4,
             Path.CURVE4,
             Path.CURVE4,
             Path.CLOSEPOLY,
             ]

    if ax == None:
        return verts, codes
    else:
        path = Path(verts, codes)
        patch = patches.PathPatch(path, facecolor=color+(0.5,), edgecolor=color+(0.4,), lw=LW)
        ax.add_patch(patch)


def ChordArc(start1=0, end1=60, start2=180, end2=240, radius=1.0, chordwidth=0.7, ax=None, color=(1,0,0)):
    # start, end should be in [0, 360)
    if start1 > end1:
        start1, end1 = end1, start1
    if start2 > end2:
        start2, end2 = end2, start2
    start1 *= np.pi/180.
    end1 *= np.pi/180.
    start2 *= np.pi/180.
    end2 *= np.pi/180.
    opt1 = 4./3. * np.tan((end1-start1)/ 4.) * radius
    opt2 = 4./3. * np.tan((end2-start2)/ 4.) * radius
    rchord = radius * (1-chordwidth)
    verts = [
        polar2xy(radius, start1),
        polar2xy(radius, start1) + polar2xy(opt1, start1+0.5*np.pi),
        polar2xy(radius, end1) + polar2xy(opt1, end1-0.5*np.pi),
        polar2xy(radius, end1),
        polar2xy(rchord, end1),
        polar2xy(rchord, start2),
        polar2xy(radius, start2),
        polar2xy(radius, start2) + polar2xy(opt2, start2+0.5*np.pi),
        polar2xy(radius, end2) + polar2xy(opt2, end2-0.5*np.pi),
        polar2xy(radius, end2),
        polar2xy(rchord, end2),
        polar2xy(rchord, start1),
        polar2xy(radius, start1),
        ]

    codes = [Path.MOVETO,
             Path.CURVE4,
             Path.CURVE4,
             Path.CURVE4,
             Path.CURVE4,
             Path.CURVE4,
             Path.CURVE4,
             Path.CURVE4,
             Path.CURVE4,
             Path.CURVE4,
             Path.CURVE4,
             Path.CURVE4,
             Path.CURVE4,
             ]

    if ax == None:
        return verts, codes
    else:
        path = Path(verts, codes)
        patch = patches.PathPatch(path, facecolor=color+(0.5,), edgecolor=color+(0.4,), lw=LW)
        ax.add_patch(patch)

def selfChordArc(start=0, end=60, radius=1.0, chordwidth=0.7, ax=None, color=(1,0,0)):
    # start, end should be in [0, 360)
    if start > end:
        start, end = end, start
    start *= np.pi/180.
    end *= np.pi/180.
    opt = 4./3. * np.tan((end-start)/ 4.) * radius
    rchord = radius * (1-chordwidth)
    verts = [
        polar2xy(radius, start),
        polar2xy(radius, start) + polar2xy(opt, start+0.5*np.pi),
        polar2xy(radius, end) + polar2xy(opt, end-0.5*np.pi),
        polar2xy(radius, end),
        polar2xy(rchord, end),
        polar2xy(rchord, start),
        polar2xy(radius, start),
        ]

    codes = [Path.MOVETO,
             Path.CURVE4,
             Path.CURVE4,
             Path.CURVE4,
             Path.CURVE4,
             Path.CURVE4,
             Path.CURVE4,
             ]

    if ax == None:
        return verts, codes
    else:
        path = Path(verts, codes)
        patch = patches.PathPatch(path, facecolor=color+(0.5,), edgecolor=color+(0.4,), lw=LW)
        ax.add_patch(patch)

def chordDiagram(X, ax, colors=None, width=0.1, pad=2, chordwidth=0.7):
    """Plot a chord diagram
    Parameters
    ----------
    X :
        flux data, X[i, j] is the flux from i to j
    ax :
        matplotlib `axes` to show the plot
    colors : optional
        user defined colors in rgb format. Use function hex2rgb() to convert hex color to rgb color. Default: d3.js category10
    width : optional
        width/thickness of the ideogram arc
    pad : optional
        gap pad between two neighboring ideogram arcs, unit: degree, default: 2 degree
    chordwidth : optional
        position of the control points for the chords, controlling the shape of the chords
    """
    # X[i, j]:  i -> j
    x = X.sum(axis = 1) # sum over rows
    ax.set_xlim(-1.1, 1.1)
    ax.set_ylim(-1.1, 1.1)

    if colors is None:
        if len(x) <= 10:
            # use d3.js category10 https://github.com/d3/d3-3.x-api-reference/blob/master/Ordinal-Scales.md#category10
            colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd',
                      '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf']
        elif len(x) <= 20:
            # use d3.js category20 https://github.com/d3/d3-3.x-api-reference/blob/master/Ordinal-Scales.md#category20
            colors = ['#1f77b4', '#aec7e8', '#ff7f0e', '#ffbb78', '#2ca02c', '#98df8a', '#d62728', '#ff9896',
                      '#9467bd', '#c5b0d5', '#8c564b', '#c49c94', '#e377c2', '#f7b6d2', '#7f7f7f', '#c7c7c7',
                      '#bcbd22', '#dbdb8d','#17becf', '#9edae5']
        else:
#             print('x is too large! Use x smaller than 10')
            print('x is too large! Use x up to 20')
        
        colors = [hex2rgb(colors[i]) for i in range(len(x))]

    # find position for each start and end
    y = x/np.sum(x).astype(float) * (360 - pad*len(x))

    pos = {}
    arc = []
    nodePos = []
    start = 0
    for i in range(len(x)):
        end = start + y[i]
        arc.append((start, end))
        angle = 0.5*(start+end)
        #print(start, end, angle)
        if -30 <= angle <= 210:
            angle -= 90
        else:
            angle -= 270
        nodePos.append(tuple(polar2xy(1.1, 0.5*(start+end)*np.pi/180.)) + (angle,))
        z = (X[i, :]/x[i].astype(float)) * (end - start)
        ids = np.argsort(z)
        z0 = start
        for j in ids:
            pos[(i, j)] = (z0, z0+z[j])
            z0 += z[j]
        start = end + pad

    for i in range(len(x)):
        start, end = arc[i]
        IdeogramArc(start=start, end=end, radius=1.0, ax=ax, color=colors[i], width=width)
        start, end = pos[(i,i)]
        selfChordArc(start, end, radius=1.-width, color=colors[i], chordwidth=chordwidth*0.7, ax=ax)
#         selfChordArc(start, end, radius=1.-width, color=(0, 0, 0), chordwidth=chordwidth*0.7, ax=ax)
        
        #posso mudar aqui para corrigir a cor da linha em relação a proporção do FROM and TO
        for j in range(i):
            color = colors[i]
            if X[i, j] > X[j, i]:
                color = colors[j]
            start1, end1 = pos[(i,j)]
            start2, end2 = pos[(j,i)]
            ChordArc(start1, end1, start2, end2,
#                      radius=1.-width, color=colors[i], chordwidth=chordwidth, ax=ax)
                     radius=1.-width, color=(0, 0, 0), chordwidth=chordwidth, ax=ax)

    #print(nodePos)
    return nodePos

# ##################################
# if __name__ == "__main__":
#     fig = plt.figure(figsize=(6,6))
#     flux = np.array([[11975,  5871, 8916, 2868],
#       [ 1951, 10048, 2060, 6171],
#       [ 8010, 16145, 8090, 8045],
#       [ 1013,   990,  940, 6907]
#     ])

#     ax = plt.axes([0,0,1,1])

#     #nodePos = chordDiagram(flux, ax, colors=[hex2rgb(x) for x in ['#666666', '#66ff66', '#ff6666', '#6666ff']])
#     nodePos = chordDiagram(flux, ax)
#     ax.axis('off')
#     prop = dict(fontsize=16*0.8, ha='center', va='center')
#     nodes = ['non-crystal', 'FCC', 'HCP', 'BCC']
#     for i in range(4):
#         ax.text(nodePos[i][0], nodePos[i][1], nodes[i], rotation=nodePos[i][2], **prop)

# #     plt.savefig("example.png", dpi=600,
# #             transparent=True,
# #             bbox_inches='tight', pad_inches=0.02)

Covid_dict é um dicinário com o mapping de cada coluna com o atributo que foi presente para um dado paciente no dataset. Possui 549 atributos nessa configuração.

In [5]:
rs = {0: 'BAGE - R22',
 1: 'PASSO FUNDO - R17 R18 R19',
 2: 'SANTA MARIA - R01 R02',
 3: 'IJUI - R13',
 4: 'SANTA ROSA - R14',
 5: 'URUGUAIANA - R03',
 6: 'PALMEIRA DAS MISSOES - R15 R20',
 7: 'CAXIAS DO SUL - R23 R24 R25 R26',
 8: 'PORTO ALEGRE - R10',
 9: 'PELOTAS - R21',
 10: 'LAJEADO - R29 R30',
 11: 'GUAIBA - R09',
 12: 'NOVO HAMBURGO - R07',
 13: 'ERECHIM - R16',
 14: 'CAPAO DA CANOA - R04 R05',
 15: 'CACHOEIRA DO SUL - R27',
 16: 'CANOAS - R08',
 17: 'CRUZ ALTA - R12',
 18: 'SANTO ANGELO - R11',
 19: 'TAQUARA - R06',
 20: 'SANTA CRUZ DO SUL - R28',
 21: 'Bebê',
 22: 'Criança',
 23: 'Jovem',
 24: 'Adulto_1',
 25: 'Adulto_2',
 26: 'Adulto_3',
 27: 'Idoso_1',
 28: 'Idoso_2',
 29: 'Idoso_3',
 30: 'Março',
 31: 'Abril',
 32: 'Maio',
 33: 'Junho',
 34: 'Julho',
 35: 'Agosto',
 36: 'Setembro',
 37: 'Outubro',
 38: 'Novembro',
 39: 'Dezembro',
 40: 'Obito_Março',
 41: 'Obito_Abril',
 42: 'Obito_Maio',
 43: 'Obito_Junho',
 44: 'Obito_Julho',
 45: 'Obito_Agosto',
 46: 'Obito_Setembro',
 47: 'Obito_Outubro',
 48: 'Obito_Novembro',
 49: 'Obito_Dezembro',
 50: 'SexF',
 51: 'SexM',
 52: 'Hospitalizado',
 53: 'Hosp_SRAG',
 54: 'Febre',
 55: 'Tosse',
 56: 'Garganta',
 57: 'Dispneia',
 58: 'Gestante',
 59: 'Asma',
 60: 'DRespiratoria',
 61: 'DNeurologica',
 62: 'Diabetes',
 63: 'Imunodef',
 64: 'DRenal',
 65: 'DHepatica',
 66: 'DCardiaca',
 67: 'Obesidade',
 68: 'DHematologica',
 69: 'Sindrome_down',
 70: '0-4',
 71: '5-14',
 72: '15-24',
 73: '25-34',
 74: '35-44',
 75: '45-54',
 76: '55-64',
 77: '65-74',
 78: '75-84',
 79: '85-94',
 80: '95-104',
 81: '105-114',
 82: '115-124',
 83: '125-134',
 84: '135-144',
 85: '145-154',
 86: '155-164',
 87: '165-174',
 88: '175-184'}

In [ ]:
sc = {0: 'GRANDE FLORIANOPOLIS',
 1: 'FOZ DO RIO ITAJAI',
 2: 'PLANALTO NORTE',
 3: 'SERRA CATARINENSE',
 4: 'XANXERE',
 5: 'NORDESTE',
 6: 'ALTO VALE DO ITAJAI',
 7: 'CARBONIFERA',
 8: 'EXTREMO SUL CATARINENSE',
 9: 'MEDIO VALE DO ITAJAI',
 10: 'LAGUNA',
 11: 'OESTE',
 12: 'ALTO VALE DO RIO DO PEIXE',
 13: 'EXTREMO OESTE',
 14: 'MEIO OESTE',
 15: 'ALTO URUGUAI CATARINENSE',
 16: 'Bebê',
 17: 'Criança',
 18: 'Jovem',
 19: 'Adulto_1',
 20: 'Adulto_2',
 21: 'Adulto_3',
 22: 'Idoso_1',
 23: 'Idoso_2',
 24: 'Idoso_3',
 25: 'Março',
 26: 'Abril',
 27: 'Maio',
 28: 'Junho',
 29: 'Julho',
 30: 'Agosto',
 31: 'Setembro',
 32: 'Outubro',
 33: 'Novembro',
 34: 'Dezembro',
 35: 'SexF',
 36: 'SexM',
 37: 'Hospitalizado',
 38: 'Hosp_SRAG',
 39: 'Febre',
 40: 'Tosse',
 41: 'Garganta',
 42: 'Dispneia',
 43: 'Gestante',
 44: 'Asma',
 45: 'DRespiratoria',
 46: 'DNeurologica',
 47: 'Diabetes',
 48: 'Imunodef',
 49: 'DRenal',
 50: 'DHepatica',
 51: 'DCardiaca',
 52: 'Obesidade',
 53: 'DHematologica',
 54: 'Sindrome_down',
 55: '0-4',
 56: '5-14',
 57: '15-24',
 58: '25-34',
 59: '35-44',
 60: '45-54',
 61: '55-64',
 62: '65-74',
 63: '75-84',
 64: '85-94',
 65: '95-104',
 66: '105-114',
 67: '115-124',
 68: '125-134',
 69: '135-144',
 70: '145-154',
 71: '155-164',
 72: '165-174',
 73: '175-184',
 74: '185-194',
 75: '195-204',
 76: '205-214',
 77: '215-224'}